# 앙상블 

- 여러 머신러닝 모델을 결합하여 더 좋은 모델을 얻는 방법 
- 앙상블의 종류

    1. 보팅(Voting)
        - 여러 개의 분류기가 투표를 통해 최종 예측 결과를 결정 
        - 종류
            - 하드 보팅 (Hard Voting)
                - 다수의 분류기가 예측한 결과값을 최종 결과로 선정 (다수결)
            - 소프트 보팅 (Soft Voting)
                - 모든 분류기가 예측한 결정 확률의 평균이 가장 높은 결과값을 최종 결과로 선정 
                - 목소리가 큰 것의 손을 들어줌.
                - --> 얘가 좀 더 성능이 잘 나오는 경향
                
    <img src = 'voting.png'>
    
    <br><br>
    
    2. 배깅(Bagging)
        - 데이터 샘플링(Bootstrap)을 통해 모델을 학습시키고 결과를 집계하는 방법 
            - Bootstrap 방식이란 
                - 데이터가 조금씩은 편향되도록 샘플링하는 기법 (복원 추출)
                - 분산이 높은 모델의 과대적합 위험을 줄이는 효과가 있음. (의사결정나무의 단점 보완)
                    - 일부러 편향을 다양하게 시켜 그것들을 합치기 때문에
        - 모두 같은 유형의 알고리즘 기반의 분류기 사용해야 함.
            - 보통 의사결정트리 모델을 백개 쯤 합침 --> RandomForest
        - 데이터 분할 시 중복 허용 (복원 추출)
        - 예시) 랜덤포레스트(RandomForest)
            - 과대적합되기 쉬운 의사결정나무의 과대적합을 줄여 성능을 높일 수 있음. 
            - 의사결정나무 모델에서 쓰이는 메소드들을 쓸 수 있음. 
                
    <img src = 'bagging.png'>
    
    <br><br>
    
    3. 부스팅(Boosting)
        - 여러 개의 분류기가 "순차적으로" 학습 수행
        - 이전 분류기의 예측이 틀린 데이터에 대해 올바르게 예측할 수 있도록 다음 분류기에게 가중치(weight)를 부여하며 학습과 예측 진행 
        - 계속하여 분류기에게 가중치를 부스팅하며 학습을 진행하기 때문에 "부스팅 방식"이라고 불림. 
        - 일반적으로 부스팅 방식이 배깅 방식에 비해 성능이 좋지만 속도가 느리고 과적합이 발생할 가능성이 더 높음 
        - 예시 : XGBoost, LightGBM
        
        <img src = "boosting.png">
       

- 첫번째가 잘맞추는 영역, 두번째가 잘맞추는 영역 ... 다 합침 

- 배깅과 부스팅의 차이
    - 부스팅은 순서대로 학습되어야 함. (직렬 연결) 

# 랜덤포레스트

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate # 교차검증 
from sklearn.ensemble import (RandomForestClassifier, 
                              GradientBoostingClassifier, 
                              ExtraTreesClassifier)

In [2]:
df = pd.read_csv('wine.csv')
df.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
x = df[['alcohol', 'sugar', 'pH']]
y = df['class']

x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                   test_size=0.2,
                                                   stratify=y,
                                                   random_state=4)

x_train.shape, x_test.shape

((5197, 3), (1300, 3))

In [6]:
# 기본적으로 100개 결정트리 사용
# 100개 동시 학습 가능 (병렬적 학습)
rf = RandomForestClassifier(
            random_state=4,
            n_jobs=-1) # 동시 몇개까지 학습시킬건지 (최대치로)
            # cpu 감당할 수 있는데까지

In [7]:
# 계산만 하는거지 학습시키는 건 아님 
scores = cross_validate(rf, 
                        x_train, 
                        y_train, 
                        return_train_score=True, 
                        # 원래는 테스트셋에 대해서만 결과 반환 
                        n_jobs=-1)

- k겹 교차 검증 생각하면 됨. 

In [8]:
scores

{'fit_time': array([0.28568006, 0.28618741, 0.28568006, 0.38924003, 0.39125991]),
 'score_time': array([0.1229105 , 0.10940433, 0.1229105 , 0.10827279, 0.10827279]),
 'test_score': array([0.87980769, 0.89326923, 0.89124158, 0.88161694, 0.90086622]),
 'train_score': array([0.99783498, 0.99807554, 0.9978355 , 0.9978355 , 0.997114  ])}

- fit_time: 다섯 모델에 대한 훈련 시간 
- score_time: 테스트 데이터에 대해 추론하는 시간
- test_score: 테스트 데이터에 대한 점수
- train_score: 훈련 데이터에 대한 점수 

In [9]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.997739101034747 0.8893603316798696


- 랜덤포레스트는 의사결정나무의 앙상블이기 때문에 의사결정나무에서 제공하는 주요 매개변수는 모두 사용 가능 
    - criterion, max_depth 등 
    - 특성 중요도 계산도 가능 
        - 랜덤포레스트의 특성 중요도는 각 의사결정나무의 특성 중요도 취합 

In [10]:
rf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=4)

In [11]:
print(rf.feature_importances_)

[0.23910783 0.49363618 0.26725599]


- 그냥 의사결정나무 하나만 사용했을 때보다 sugar 특성의 중요도가 낮아짐 (<-- 0.6)
- alcohol과 pH 특성의 중요도는 상승함. --> "일반화 성능이 상승함" 
- --> bagging 때문임. 
    - 일부로 조금조금 편향되게 골라냈기 때문에 
    - sugar 사용하지 않고도 red wine, white wine 분류하는 법을 배운 것 
    - 랜덤 포레스트는 하나의 특성에 과도하게 집중되지 않고 더 많은 특성이 훈련에 기여할 기회를 얻음.
    - 과대적합이 줄어들고 일반화 성능을 높일 수 있음. 

In [12]:
rf = RandomForestClassifier(oob_score=True,
                           n_jobs=-1,
                           random_state=4)

rf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=4)

- out-of-bag score (oob_score) 
    - 복원 추출해서 뽑는데 (bootstrap) 이렇게 나무 100개 만들어낸다 하더라도,
    - 일부 데이터는 한번도 선택되지 않았을 수 있음 --> "out-of-bag" 샘플 
    - 이 친구들 또한 test data 와 다름 없는 애들이 됨. 
    - 일반화 성능을 측정하는데 하나의 지표가 될 수 있음. 
        - 데이터가 적은 상황에서 train/test 나누지도 못할 상황에서 사용 가능 

In [13]:
print(rf.oob_score_) 
# cross_validate 한 점수와 비슷

0.8982105060611891


# 엑스트라 트리 

- 랜덤 포레스트와 매우 유사 
    - 기본적으로 100개의 의사결정나무 훈련
    - 의사결정나무가 제공하는 대부분의 매개변수 지원
    
- 랜덤 포레스트와의 차이점
    - 부트스트랩 샘플을 사용하지 않음.
        - 전체 훈련세트 사용 
    - 노드를 분할할 때 가장 좋은 분할을 찾는 게 아니라 무작위로 분할 
        - 하나의 의사결정나무에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만
        - 많은 트리를 앙상블하기 때문에 과대적합을 방지하고 검증세트 점수를 높이는 효과가 나타남.
        - (+) 계산속도가 더 빠름. 

In [17]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=4)
scores = cross_validate(et, x_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.997739101034747 0.8889738654031243


- 효과 볼려면 나무 수 많거나, 더 자세히 학습을 해야함.
- 완전 기본 상태의 엑스트라 트리는 성능 자체는 보통 좀더 떨어짐. 
- 예제에서는 독립변수가 많지 않아서 랜덤포레스트와의 차이가 크지 않음.
- 일반적으로 엑스트라 트리가 무작위성이 더 크기 때문에 랜덤포레스트보다 더 많은 트리를 훈련해야함. 
- 하지만 랜덤하게 노드를 분할하기 때문에 계산속도는 더 빠름. 
    - 정확도는 비슷하더라도 시간은 더 벌 수 있음. 

In [19]:
# 엑스트라 트리도 특성 중요도 제공 
et.fit(x_train, y_train)

et.feature_importances_

array([0.20237273, 0.5168859 , 0.28074137])

- 하나의 의사결정나무 보다는 일반화 성능이 더 좋음. 

# 그래디언트 부스팅

- 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 기법 
- 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정트리를 100개 사용 
    - 깊이가 얕은 결정트리를 사용하여 과대적합을 방지할 수 있고 높은 일반화 성능을 기대 가능 
- 앞의 모델의 오차 예측하여, 예측값 수정. 또 그전까지의 모델의 결과의 오차를 예측... 이렇게 오차를 줄여나가는 모델 계속.. 

In [21]:
gb = GradientBoostingClassifier(random_state=4) # 부스팅에는 n_jobs 존재 불가
# 병렬적으로 연산이 불가능하고, 무조건 직렬 이기 때문 !! 
scores = cross_validate(gb, x_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8852225474789824 0.8666561782779298


- 그래디언트 부스팅은 결정트리의 개수 늘려도 과대적합에 강함.
- 원래 그냥 부스팅은 과대적합의 위험이 있지만, GradientBoosting은 강함.
    - 깊이 얕은 결정트리를 여러개 사용해서
    - 더 많이 이어붙여도 과대적합 발생 가능성 적음 

In [22]:
gb = GradientBoostingClassifier(random_state=4,
                               n_estimators=500, # 트리 개수 더 많이
                               learning_rate=0.2) # 오차 예측한 걸 얼마나 반영해서 고쳐나갈 것인가 
                                # 디폴트는 0.1
scores = cross_validate(gb, x_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9437655707561889 0.8712747094099356


In [23]:
gb.fit(x_train, y_train)
gb.feature_importances_

array([0.16530862, 0.67028501, 0.16440637])